In [2]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value
import random

In [3]:
# Sample data (replace with your actual data)
num_people = 5
num_modes = 3
num_nodes = 4

# Adjusted Capacity and Availability of the transportation modes to make the problem feasible
Capacity = {0: 8, 1: 10, 2: 15}
Availability = {0: 1, 1: 1, 2: 1}

# Sample data with adjusted Capacity and Availability
Time = {(i, m, a, b): i * 10 + m * 5 + a * 3 + b * 2 for i in range(num_people) for m in range(num_modes) for a in range(num_nodes) for b in range(num_nodes)}
Demand = {i: 1 for i in range(num_people)}
TrafficCongestion = {(m, a, b): random.uniform(1.0, 2.0) for m in range(num_modes) for a in range(num_nodes) for b in range(num_nodes)}

In [4]:
# Create the LP problem
model = LpProblem(name="EvacuationProblem", sense=LpMinimize)

# Decision variables
x = {(i, j): LpVariable(name=f"x_{i}_{j}", cat="Binary") for i in range(num_people) for j in range(num_modes)}

# Objective function with traffic simulation
print(x)
for i in range(num_people):
    for j in range(num_modes):
        for a in range(num_modes):
            for b in range(num_modes):
                model += lpSum((Time[i, j, a, b] * TrafficCongestion[j, a, b]) * x[i, j]), "TotalEvacuationTime"


{(0, 0): x_0_0, (0, 1): x_0_1, (0, 2): x_0_2, (1, 0): x_1_0, (1, 1): x_1_1, (1, 2): x_1_2, (2, 0): x_2_0, (2, 1): x_2_1, (2, 2): x_2_2, (3, 0): x_3_0, (3, 1): x_3_1, (3, 2): x_3_2, (4, 0): x_4_0, (4, 1): x_4_1, (4, 2): x_4_2}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [5]:
# Capacity constraints
for j in range(num_modes):
    model += lpSum(Demand[i] * x[i, j] for i in range(num_people)) <= Capacity[j], f"CapacityConstraint_{j}"

# Availability constraints
for j in range(num_modes):
    model += lpSum(Demand[i] * x[i, j] for i in range(num_people)) <= Availability[j], f"AvailabilityConstraint_{j}"

# Assignment constraints
for i in range(num_people):
    model += lpSum(x[i, j] for j in range(num_modes)) == 1, f"AssignmentConstraint_{i}"

# Mode availability constraints
for i in range(num_people):
    for j in range(num_modes):
        if Availability[j] == 0:
            model += x[i, j] == 0, f"ModeAvailabilityConstraint_{i}_{j}"

In [6]:
# Solve the problem
model.solve()

# Display the results
print("Status:", value(model.status))
print("Total Evacuation Time:", value(model.objective))
print("Assignment:")
for i in range(num_people):
    for j in range(num_modes):
        if value(x[i, j]) == 1:
            print(f"Person {i} uses Mode {j}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/xs/5nydm67x3fj41732cnf_d1nw0000gq/T/182594cc0ee34dbc9e3969865cdf36a9-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/xs/5nydm67x3fj41732cnf_d1nw0000gq/T/182594cc0ee34dbc9e3969865cdf36a9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 93 RHS
At line 105 BOUNDS
At line 121 ENDATA
Problem MODEL has 11 rows, 15 columns and 45 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

Status: -1
Total Evacuation Time: 0.0
Assignment:
Person 0 uses Mode 2
Person 1 uses Mode 2
Person 2 uses Mode 0
Person 3 uses 

In [ ]:
# how many people do you want to transfer
# are the roads open or not?

#presentation to the 8th army
# show through a demonstration that we can predict what will happen with the model, use diffeent reasons
# show that you can make different decisions based on this, or that the neo will last longer if you use that
# dont run the code live, show screenshots
# maybe have the figma working!! 

# you can add condition checks to avoid


# what would be really attractive to the 8th army is to show if the biggest hub is obselete. what would
#    you do in this situation

# ^^ get visualization of these effects

# explain under our  notional data, this is what it looked like for baseline, and this is looked like with extreme conditions
# you can run this 


'''ArithmeticError
- open/close nodes
- improve capacity of the nodes
- they want to know if we are able to figure out if a link between two nodes is disrupted, what would happen
- be able to visualize the node capacity at this time step and what they would exprct it to be at the end of the evacuation
- population density:
---> the numbers they provided are the state department numbers. it is a huge assumtion -> is there any way to parameterize the
     the number of people to come to a certain node at a certain time. they noticed this in Afghanistan (they assumed a certain
     number of people enter))
-> what goes into the total time of NEO
--> in showing the demo, create a visual of how the total time is calculated
----> show that this mathematical model is really scalable. when it comes to turning on and off a node, we CAN account for that

- do a "what if" analysis. you can turn a link off by turning node capacity to 0 OR turn transport time to +inf?

- change the frontend where you can add/subtract things (call it out in the presentation)
'''

